In [10]:
!pip install openpyxl

In [47]:
!pip install haversine

In [19]:
import pandas as pd

df_merge = pd.read_csv('data/데이터병합.csv', sep=',', encoding='cp949')
df_merge.head()
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283 entries, 0 to 282
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   터미널명    283 non-null    object 
 1   주소      282 non-null    object 
 2   등록일자    282 non-null    object 
 3   위도      282 non-null    float64
 4   경도      282 non-null    float64
 5   번호      283 non-null    int64  
 6   보관대수    283 non-null    int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 15.6+ KB


In [20]:
df_path = pd.read_excel('data/path/path_count4.xlsx', engine='openpyxl')
df_path.head()
df_path.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19458 entries, 0 to 19457
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   경로      19458 non-null  object
 1   new 출발  19458 non-null  int64 
 2   new 도착  19458 non-null  int64 
 3   개수      19458 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 608.2+ KB


In [73]:
df_path = df_path[df_path['개수']>=500]

In [74]:
extract_col = df_merge[['번호', '위도', '경도']]
terminal_dict = dict()
for num, lat, long in zip(extract_col['번호'], extract_col['위도'], extract_col['경도']):
    terminal_dict[num] = [lat, long]

In [75]:
samePath = []
for path in df_path['경로']:
    sp_lst = path.split('~')
    if sp_lst[0] == sp_lst[1]:
            samePath.append(sp_lst[0])

In [89]:
samePath

['222',
 '18',
 '9',
 '27',
 '263',
 '117',
 '34',
 '2',
 '8',
 '168',
 '118',
 '272',
 '28',
 '232',
 '170',
 '35',
 '121']

In [90]:
from haversine import haversine

searchNode_dict = dict()
for stdnum in samePath:
    distance = 100000
    for terminal_num in list(terminal_dict.keys()):
        # print(terminal_num)
        if terminal_num == int(stdnum):
            continue
        node1 = terminal_dict[int(stdnum)]
        node2 = terminal_dict[terminal_num]
        dist = haversine(node1, node2)
        # dist = math.sqrt(pow(node1[0] - node2[0],2) + pow(node1[1] - node2[1],2))
        if dist < distance:
            distance = dist
            result = [int(stdnum), int(terminal_num), distance]
    searchNode_dict[result[0]] = distance

In [91]:
searchNode_dict

{222: 0.5846332071373063,
 18: 0.307439196479684,
 9: 0.46503919959941586,
 27: 0.350042372185881,
 263: 0.3293266369093569,
 117: 0.6852010417974724,
 34: 0.13161935631874555,
 2: 0.23985400518447553,
 8: 0.5340404428375524,
 168: 0.6852010417974724,
 118: 0.1910672085922716,
 272: 0.6410664538477558,
 28: 0.24544707424848694,
 232: 0.3856843717086211,
 170: 0.4315212557060822,
 35: 0.4857682794887307,
 121: 0.46503919959941586}

In [92]:
import folium
from folium.plugins import MarkerCluster

df_merge = df_merge.dropna(axis=0)

m = folium.Map(
    location=[35.2538, 128.6402],
    zoom_start=11
)

coords1 = df_merge[['번호','위도', '경도']]


marker_cluster1 = MarkerCluster().add_to(m)

for name, lat, long in zip(coords1['번호'], coords1['위도'], coords1['경도']):
    folium.Marker([lat, long], popup= [name], icon = folium.Icon(color="green")).add_to(marker_cluster1)
    if str(name) in samePath:
        folium.Circle([lat, long], radius = searchNode_dict[name] * 800).add_to(m)
m

